In [9]:
!pip install osmnx
!pip install pandas
!pip install folium
!pip install scikit-learn

In [10]:
import osmnx as ox
import pandas as pd
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [11]:
def is_bike_friendly(edge):
    # Criteria for bike-friendly paths
    bike_friendly_tags = ['yes', 'designated', 'permissive', 'lane', 'shared', 'shared_lane']
    bike_path_types = ['cycleway', 'path', 'living_street', 'residential']

    # Check for bicycle tag or highway type
    is_bike_tag_friendly = edge.get('bicycle') in bike_friendly_tags
    is_highway_type_friendly = edge.get('highway') in bike_path_types

    # Check for specific infrastructure for bikes
    has_cycleway_infrastructure = ('cycleway' in edge) or ('cycleway:right' in edge) or ('cycleway:left' in edge)

    # Additional check for footways and pedestrian paths if explicitly marked as bike-friendly
    is_footway_bike_friendly = (edge.get('highway') in ['footway', 'pedestrian']) and edge.get('bicycle') == 'yes'

    return is_bike_tag_friendly or is_highway_type_friendly or has_cycleway_infrastructure or is_footway_bike_friendly

place_name = "Münster, Germany"
G = ox.graph_from_place(place_name, network_type='all')
df = pd.read_csv('filtered_data_munster.csv')

# Identify bus stops
tags_bus = {'highway': 'bus_stop'}
bus_stops = ox.geometries_from_place(place_name, tags=tags_bus)

# Filter bike-friendly edges
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)]

# Convert to a projected CRS before creating buffers
crs_proj = 'EPSG:32632'  # UTM zone 32N, suitable for Germany
bus_stops = bus_stops.to_crs(crs_proj)
bike_friendly_edges = bike_friendly_edges.to_crs(crs_proj)
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs(crs_proj)

# Create 20 meter buffers
bus_stops_buffer = bus_stops.buffer(20)  # For bus stops
bike_friendly_buffer = bike_friendly_edges.buffer(20)  # For bike paths

# Identify nodes near bus stops and bike paths
near_bus_stops = nodes[nodes.geometry.apply(lambda x: bus_stops_buffer.contains(x).any())]
near_bike_paths = nodes[nodes.geometry.apply(lambda x: bike_friendly_buffer.contains(x).any())]
near_both = near_bus_stops[near_bus_stops.index.isin(near_bike_paths.index)].to_crs('EPSG:4326')  # Convert back to geographic CRS

# Create the map with Folium
map = folium.Map(location=[51.9625, 7.6256], zoom_start=13)

df_near_both = pd.DataFrame([(point.y, point.x) for point in near_both.geometry], columns=['Latitude', 'Longitude'])

num_clusters = 10  # Adjust the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(df_near_both[['Latitude', 'Longitude']])

# Add the cluster labels to your DataFrame
df_near_both['Cluster'] = clusters

# Generate a color palette for the clusters
colors = plt.cm.tab10.colors[:num_clusters]  # Using a matplotlib colormap

# Use different colors for each cluster
num_clusters = 10  # Adjust the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(df_near_both[['Latitude', 'Longitude']])

# Add the cluster labels to your DataFrame
df_near_both['Cluster'] = clusters

# Create a colormap
colormap = cm.get_cmap('tab10', num_clusters)  # You can choose different colormaps

# Use different colors for each cluster
for cluster_id in range(num_clusters):
    cluster_data = df_near_both[df_near_both['Cluster'] == cluster_id]
    color = colormap(cluster_id)
    for _, row in cluster_data.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,
            color=cm.colors.rgb2hex(color),
            fill=True,
            fill_color=cm.colors.rgb2hex(color),
            fill_opacity=0.7
        ).add_to(map)

# Add markers for additional data from df
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='blue',
        fill=True
    ).add_to(map)

# Save and display the map
map.save('k_analize_munster.html')
map

/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_42044/869524782.py:24: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bus_stops = ox.geometries_from_place(place_name, tags=tags_bus)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_para